In [1]:
import numpy as np
import pandas as pd
import string
import calendar
import re
import math
import sympy
import matplotlib.pyplot as plt
import pprint
import itertools

from fractions import Fraction
from scipy import stats
from scipy import special


print(f"numpy version is {np.__version__}")
print(f"pandas version is {pd.__version__}")


numpy version is 1.20.3
pandas version is 1.3.2


### $\overline{x}$ control chart

In [122]:
rawD        = pd.DataFrame(np.loadtxt('xBar_control_chart.txt', delimiter=' ', usecols=[1, 3]).flatten(order='F'), columns=['xBar'])
k           = rawD.shape[0]
n           = 5

# historic data
mu          = 35
sigma       = 3
LCL, UCL    = mu - 3*sigma/math.sqrt(n), mu + 3*sigma/math.sqrt(n)

violators   = rawD.loc[(rawD['xBar'] > UCL) | (rawD['xBar'] < LCL)]
inControl   = rawD.loc[(rawD['xBar'] < UCL) & (rawD['xBar'] > LCL)]

# strings of coordinates for scatter plot

coords_1     = list(zip(inControl.index + 1, inControl['xBar']))
cd_str_1     = ' '.join(map(str, coords_1))

coords_2     = list(zip(violators.index + 1, violators['xBar']))
cd_str_2     = ' '.join(map(str, coords_2))

### $\overline{x}$ control chart using estimated $\mu, \sigma$

In [81]:
# calculated data

k_init          = 50
n               = 6
cn              = math.sqrt(2/(n-1)) * special.gamma(n/2) / special.gamma(n/2 - 1/2)
sample_mean     = 970 / k_init
sample_std      = 85 / k_init



mu_est          = sample_mean
sigma_est       = sample_std / cn

print(f'Sample grand mean : {mu_est} and Sample stddev : {sigma_est}')

LCL, UCL        = mu_est - 3*sigma_est/math.sqrt(n), mu_est + 3*sigma_est/math.sqrt(n)
# LCL, UCL        = max(sigma_est * cn - 3*sigma_est * math.sqrt(1 - cn*cn), 0), sigma_est * cn + 3*sigma_est * math.sqrt(1 - cn*cn)
print(f'LCL = {LCL} and UCL = {UCL}')

process_ulim, process_llim      = 19+4, 19-4
active_dist     = stats.norm(loc = mu_est, scale=sigma_est)
pass_prob       = active_dist.cdf(process_ulim) - active_dist.cdf(process_llim)
print(f'Net probability of aprroval = {pass_prob}')

# print(active_dist.cdf(400))


Sample grand mean : 19.4 and Sample stddev : 1.7865909502268398
LCL = 17.211881896435052 and UCL = 21.588118103564945
Net probability of aprroval = 0.9711552257659718


### preprocess raw data and write file

In [71]:
rawD            = pd.read_csv('control_chart_rawData.txt', delimiter = ' ', header = None)
rawD.iloc[:, 1:]   = rawD.iloc[:, 1:] * 0.01 + 0.25
rawD_means      = rawD.iloc[:, 1:].mean(axis = 1)
rawD_std        = rawD.iloc[:, 1:].std(axis = 1)


save_rawD       = pd.DataFrame({'Subgroup' : rawD.iloc[:, 0], 'xBar' : rawD_means, 'S' : rawD_std})
save_rawD.to_csv('xBar_control_chart.txt', header = False, sep = ' ', index = False)

### iterative $\overline{x}$ or S control chart

In [77]:
rawNP           = pd.DataFrame(np.reshape(np.loadtxt('xBar_control_chart.txt', delimiter=' '), (-1, 3), order='C'), columns=['Subgroup', 'xBar', 'S'])
rawNP           = rawNP.sort_values(by = ['Subgroup'])

# trimming violators iteratively
# rawNP           = rawNP[~((rawNP.Subgroup == 10.0) | (rawNP.Subgroup == 15.0))]

k_init          = rawNP.shape[0]
n               = 5
cn              = math.sqrt(2/(n-1)) * special.gamma(n/2) / special.gamma(n/2 - 1/2)

mu_est          = rawNP['xBar'].mean()
sigma_est       = rawNP['S'].mean() / cn
print(mu_est, sigma_est)

# probability of approval within range

# process_ulim, process_llim      = 35.3+10, 35.3-10
# active_dist     = stats.norm(loc = mu_est, scale=sigma_est)
# pass_prob       = active_dist.cdf(process_ulim) - active_dist.cdf(process_llim)
# print(f'Net probability of aprroval = {pass_prob}')

# X bar control chart

LCL, UCL        = mu_est - 3*sigma_est/math.sqrt(n), mu_est + 3*sigma_est/math.sqrt(n)
print(f'LCL = {LCL} and UCL = {UCL}')
violators       = rawNP.loc[(rawNP['xBar'] > UCL) | (rawNP['xBar'] < LCL)]
inControl       = rawNP.loc[(rawNP['xBar'] < UCL) & (rawNP['xBar'] > LCL)]
coords_1     = list(zip(inControl['Subgroup'], inControl['xBar']))
cd_str_1     = ' '.join(map(str, coords_1))

coords_2     = list(zip(violators['Subgroup'], violators['xBar']))
cd_str_2     = ' '.join(map(str, coords_2))

# S control chart

# LCL, UCL        = max(sigma_est * cn - 3*sigma_est * math.sqrt(1 - cn*cn), 0), sigma_est * cn + 3*sigma_est * math.sqrt(1 - cn*cn)
# print(f'LCL = {LCL} and UCL = {UCL}')
# violators       = rawNP.loc[(rawNP['S'] > UCL) | (rawNP['S'] < LCL)]
# inControl       = rawNP.loc[(rawNP['S'] < UCL) & (rawNP['S'] > LCL)]
# coords_1     = list(zip(inControl['Subgroup'], inControl['S']))
# cd_str_1     = ' '.join(map(str, coords_1))
# coords_2     = list(zip(violators['Subgroup'], violators['S']))
# cd_str_2     = ' '.join(map(str, coords_2))




0.2514666666666667 0.0114071587748027
LCL = 0.23616235719631148 and UCL = 0.26677097613702194


#### print figure

In [90]:
before          = '\\begin{figure}[H] \n \t \\centering \n \t \\begin{tikzpicture} \n \t \t \\begin{axis}[width = 0.75\\textwidth, xlabel = Subgroup, ylabel = $\\overline{x}$ , grid = both, grid style = {dotted, gray}]'
after           = '\t \t \\end{axis} \n \t \\end{tikzpicture} \n \\end{figure}'
str_scatter     = '\t \t \\addplot[only marks, color = blue] plot coordinates{' + cd_str_1 +'};'
str_viol        = '\t \t \\addplot[only marks, color = red, mark size = 2pt] plot coordinates{' + cd_str_2 +'};'
UCL_line        = '\t \t \\addplot[mark = none, line width = 1pt, color = ForestGreen, domain = 0:' + str(k_init+1) + ']{' + str(round(UCL, 2)) + '} node[below, pos = 0.5] {\\texttt{UCL = ' + str(round(UCL, 4)) +'}};'
LCL_line        = '\t \t \\addplot[mark = none, line width = 1pt, color = ForestGreen, domain = 0:' + str(k_init+1) + ']{' + str(round(LCL, 2)) + '} node[above, pos = 0.5] {\\texttt{LCL = ' + str(round(LCL, 4)) +'}};'

print(before,'\n',str_scatter, '\n',str_viol, '\n',LCL_line, '\n',UCL_line, '\n', after)

\begin{figure}[H] 
 	 \centering 
 	 \begin{tikzpicture} 
 	 	 \begin{axis}[width = 0.75\textwidth, xlabel = Subgroup, ylabel = $\overline{x}$ , grid = both, grid style = {dotted, gray}] 
 	 	 \addplot[only marks, color = blue] plot coordinates{(1.0, 0.2604) (2.0, 0.2572) (3.0, 0.25739999999999996) (4.0, 0.2448) (5.0, 0.2502) (6.0, 0.25379999999999997) (7.0, 0.25060000000000004) (8.0, 0.25) (9.0, 0.2492) (10.0, 0.2502) (11.0, 0.25759999999999994) (12.0, 0.2452) (13.0, 0.2444) (14.0, 0.2448) (15.0, 0.25620000000000004)}; 
 	 	 \addplot[only marks, color = red, mark size = 2pt] plot coordinates{}; 
 	 	 \addplot[mark = none, line width = 1pt, color = ForestGreen, domain = 0:51]{17.21} node[above, pos = 0.5] {\texttt{LCL = 17.2119}}; 
 	 	 \addplot[mark = none, line width = 1pt, color = ForestGreen, domain = 0:51]{21.59} node[below, pos = 0.5] {\texttt{UCL = 21.5881}}; 
 	 	 \end{axis} 
 	 \end{tikzpicture} 
 \end{figure}


#### detecting shifts in mean

In [33]:
old_mu          = 431
old_sigma       = 24.42
n               = 4
a               = 60

p_detect        = 1 - stats.norm.cdf(3 - a * math.sqrt(n) / old_sigma)
print(f'Probability of detection is {p_detect}')
print(f'Mean number of subgroups needed = {1/p_detect}')

Probability of detection is 0.9721902360648702
Mean number of subgroups needed = 1.0286052697337253


### control charts for fraction defective

In [109]:
# numDef          = pd.DataFrame(np.reshape(np.loadtxt('num_def_control_chart.txt', delimiter=' '), (-1, 2), order='C'), columns=['Subgroup', 'Defects'], dtype = int)
numDef          = pd.DataFrame(np.reshape(np.loadtxt('num_def_control_chart.txt', delimiter=' '), (-1, 3), order='C'), columns=['Subgroup', 'n', 'Defects'], dtype = int)

numDef          = numDef.sort_values(by = ['Subgroup'])

k_def           = numDef.shape[0]
numDef['Def_Frac']  = numDef['Defects'] / numDef['n']

# trimming violators iteratively
# numDef           = numDef[~((numDef.Subgroup == 10.0) | (numDef.Subgroup == 15.0))]

F_bar           = numDef['Def_Frac'].mean()
UCL, LCL        = F_bar + 3*math.sqrt(F_bar * (1 - F_bar) / n), max(0, F_bar - 3*math.sqrt(F_bar * (1 - F_bar)))

print(f'LCL = {LCL} and UCL = {UCL}, F_bar = {F_bar}')
violators       = numDef.loc[(numDef['Def_Frac'] > UCL) | (numDef['Def_Frac'] < LCL)]
inControl       = numDef.loc[(numDef['Def_Frac'] < UCL) & (numDef['Def_Frac'] > LCL)]
coords_1     = list(zip(inControl['Subgroup'], inControl['Def_Frac']))
cd_str_1     = ' '.join(map(str, coords_1))

coords_2     = list(zip(violators['Subgroup'], violators['Def_Frac']))
cd_str_2     = ' '.join(map(str, coords_2))


LCL = 0 and UCL = 0.1484735486908886, F_bar = 0.07128397065897066


#### print figures

In [108]:
before          = '\\begin{figure}[H] \n \t \\centering \n \t \\begin{tikzpicture} \n \t \t \\begin{axis}[ylabel near ticks, width = 0.75\\textwidth, xlabel = Subgroup, ylabel = $F$ , grid = both, grid style = {dotted, gray}]'
after           = '\t \t \\end{axis} \n \t \\end{tikzpicture} \n \\end{figure}'
str_scatter     = '\t \t \\addplot[only marks, color = blue] plot coordinates{' + cd_str_1 +'};'
str_viol        = '\t \t \\addplot[only marks, color = red, mark size = 2pt] plot coordinates{' + cd_str_2 +'};'
LCL_line        = '\t \t \\addplot[mark = none, line width = 1pt, color = ForestGreen, domain = 0:' + str(k_def+1) + ']{' + str(round(LCL, 2)) + '} node[above, pos = 0.5] {\\texttt{LCL = ' + str(round(LCL, 4)) +'}};'
UCL_line        = '\t \t \\addplot[mark = none, line width = 1pt, color = ForestGreen, domain = 0:' + str(k_def+1) + ']{' + str(round(UCL, 2)) + '} node[below, pos = 0.5] {\\texttt{UCL = ' + str(round(UCL, 4)) +'}};'

print(before,'\n',str_scatter, '\n',str_viol, '\n',LCL_line, '\n',UCL_line, '\n', after)

\begin{figure}[H] 
 	 \centering 
 	 \begin{tikzpicture} 
 	 	 \begin{axis}[ylabel near ticks, width = 0.75\textwidth, xlabel = Subgroup, ylabel = $F$ , grid = both, grid style = {dotted, gray}] 
 	 	 \addplot[only marks, color = blue] plot coordinates{(1, 0.0625) (2, 0.06363636363636363) (3, 0.044444444444444446) (4, 0.1125) (5, 0.12) (6, 0.1111111111111111) (7, 0.05) (8, 0.04285714285714286) (9, 0.0625) (10, 0.06666666666666667) (11, 0.05555555555555555) (12, 0.06363636363636363)}; 
 	 	 \addplot[only marks, color = red, mark size = 2pt] plot coordinates{}; 
 	 	 \addplot[mark = none, line width = 1pt, color = ForestGreen, domain = 0:13]{0} node[above, pos = 0.5] {\texttt{LCL = 0}}; 
 	 	 \addplot[mark = none, line width = 1pt, color = ForestGreen, domain = 0:13]{0.15} node[below, pos = 0.5] {\texttt{UCL = 0.1485}}; 
 	 	 \end{axis} 
 	 \end{tikzpicture} 
 \end{figure}


#### detecting shifts in mean

In [113]:
old_p           = 0.04
n               = 500
new_p           = 0.08
PoisMean        = n*old_p

UCL, LCL        = PoisMean + 3 * math.sqrt(PoisMean), PoisMean - 3 * math.sqrt(PoisMean)
print(f'LCL = {LCL} and UCL = {UCL}')

p_detect_high        = 1 - stats.binom.cdf(UCL, n, new_p)
print(f'Upper Probability of detection is {p_detect_high}')
p_detect_low        = stats.binom.cdf(LCL, n, new_p)
print(f'Lower Probability of detection is {p_detect_low}')


LCL = 6.583592135001261 and UCL = 33.41640786499874
Upper Probability of detection is 0.8590457278988749
Lower Probability of detection is 8.251441617311019e-12


### number defective - Poisson approx

In [146]:
all_def         = pd.DataFrame(np.reshape(np.loadtxt('num_def_control_chart.txt', delimiter=' '), (-1, 2), order='C'), columns=['Subgroup', 'Defects'], dtype = int)
numDef          = numDef.sort_values(by = ['Subgroup'])
k_def           = all_def.shape[0]
# trimming step
all_def           = all_def[~((all_def.Subgroup == 14) | (all_def.Subgroup == 14))]


Xbar_est        = all_def['Defects'].mean()

UCL, LCL        = Xbar_est + 3 * math.sqrt(Xbar_est), Xbar_est - 3 * math.sqrt(Xbar_est)
print(f'LCL = {LCL} and UCL = {UCL}')

violators       = all_def.loc[(all_def['Defects'] > UCL) | (all_def['Defects'] < LCL)]
inControl       = all_def.loc[(all_def['Defects'] < UCL) & (all_def['Defects'] > LCL)]
coords_1     = list(zip(inControl['Subgroup'], inControl['Defects']))
cd_str_1     = ' '.join(map(str, coords_1))

coords_2     = list(zip(violators['Subgroup'], violators['Defects']))
cd_str_2     = ' '.join(map(str, coords_2))


LCL = -2.112486080160912 and UCL = 9.112486080160913


In [147]:
before          = '\\begin{figure}[H] \n \t \\centering \n \t \\begin{tikzpicture} \n \t \t \\begin{axis}[ylabel near ticks, width = 0.75\\textwidth, xlabel = Subgroup, ylabel = $X$ , grid = both, grid style = {dotted, gray}]'
after           = '\t \t \\end{axis} \n \t \\end{tikzpicture} \n \\end{figure}'
str_scatter     = '\t \t \\addplot[only marks, color = blue] plot coordinates{' + cd_str_1 +'};'
str_viol        = '\t \t \\addplot[only marks, color = red, mark size = 2pt] plot coordinates{' + cd_str_2 +'};'
LCL_line        = '\t \t \\addplot[mark = none, line width = 1pt, color = ForestGreen, domain = 0:' + str(k_def+1) + ']{' + str(round(LCL, 2)) + '} node[above, pos = 0.5] {\\texttt{LCL = ' + str(round(LCL, 4)) +'}};'
UCL_line        = '\t \t \\addplot[mark = none, line width = 1pt, color = ForestGreen, domain = 0:' + str(k_def+1) + ']{' + str(round(UCL, 2)) + '} node[below, pos = 0.5] {\\texttt{UCL = ' + str(round(UCL, 4)) +'}};'

print(before,'\n',str_scatter, '\n',str_viol, '\n',LCL_line, '\n',UCL_line, '\n', after)

\begin{figure}[H] 
 	 \centering 
 	 \begin{tikzpicture} 
 	 	 \begin{axis}[ylabel near ticks, width = 0.75\textwidth, xlabel = Subgroup, ylabel = $X$ , grid = both, grid style = {dotted, gray}] 
 	 	 \addplot[only marks, color = blue] plot coordinates{(1.0, 2.0) (2.0, 3.0) (15.0, 2.0) (3.0, 4.0) (16.0, 2.0) (4.0, 3.0) (17.0, 6.0) (5.0, 1.0) (18.0, 5.0) (6.0, 2.0) (19.0, 4.0) (7.0, 5.0) (20.0, 6.0) (8.0, 0.0) (21.0, 3.0) (9.0, 2.0) (22.0, 7.0) (10.0, 5.0) (23.0, 0.0) (11.0, 1.0) (24.0, 2.0) (12.0, 7.0) (25.0, 4.0) (13.0, 8.0)}; 
 	 	 \addplot[only marks, color = red, mark size = 2pt] plot coordinates{}; 
 	 	 \addplot[mark = none, line width = 1pt, color = ForestGreen, domain = 0:27]{-2.11} node[above, pos = 0.5] {\texttt{LCL = -2.1125}}; 
 	 	 \addplot[mark = none, line width = 1pt, color = ForestGreen, domain = 0:27]{9.11} node[below, pos = 0.5] {\texttt{UCL = 9.1125}}; 
 	 	 \end{axis} 
 	 \end{tikzpicture} 
 \end{figure}


### drifts in population mean

#### moving average

In [27]:
movAv               = pd.DataFrame(np.reshape(np.loadtxt('moving_average.txt', delimiter=' '), (-1, 2), order='C'), columns=['Xt_bar', 'Mt']).dropna()
k,n                 = 8, 4
meanP, stdP         = 50, math.sqrt(5)

# movAv           = movAv[~((movAv.index == 2) | (movAv.index == 4))]


movAv['Mt_UCL'], movAv['Mt_LCL']    = movAv['Xt_bar'] * 0, movAv['Xt_bar'] * 0
k_def               = movAv.shape[0]

for i in range(k_def):
    if i < k:
        movAv['Mt_UCL'][i]      = meanP + 3 * stdP / math.sqrt(n * (i+1))
        movAv['Mt_LCL'][i]      = meanP - 3 * stdP / math.sqrt(n * (i+1))
    else:
        movAv['Mt_UCL'][i]      = meanP + 3 * stdP / math.sqrt(n * k)
        movAv['Mt_LCL'][i]      = meanP - 3 * stdP / math.sqrt(n * k)

violators       = movAv.loc[(movAv['Mt'] > movAv['Mt_UCL']) | (movAv['Mt'] < movAv['Mt_LCL'])]
inControl       = movAv.loc[(movAv['Mt'] < movAv['Mt_UCL']) & (movAv['Mt'] > movAv['Mt_LCL'])]

coords_1     = list(zip(inControl.index + 1, inControl['Mt']))
cd_str_1     = ' '.join(map(str, coords_1))

coords_2     = list(zip(violators.index + 1, violators['Mt']))
cd_str_2     = ' '.join(map(str, coords_2))

coords_3     = list(zip(movAv.index +1, movAv['Mt_LCL']))
cd_str_3     = ' '.join(map(str, coords_3))

coords_4     = list(zip(movAv.index +1, movAv['Mt_UCL']))
cd_str_4     = ' '.join(map(str, coords_4))


before          = '\\begin{figure}[H] \n \t \\centering \n \t \\begin{tikzpicture} \n \t \t \\begin{axis}[ylabel near ticks, width = 0.75\\textwidth, xlabel = Subgroup, ylabel = $M_t$ , grid = both, grid style = {dotted, gray}]'
after           = '\t \t \\end{axis} \n \t \\end{tikzpicture} \n \\end{figure}'
str_scatter     = '\t \t \\addplot[only marks, color = blue] plot coordinates{' + cd_str_1 +'};'
str_viol        = '\t \t \\addplot[only marks, color = red, mark size = 2pt] plot coordinates{' + cd_str_2 +'};'
LCL_line        = '\t \t \\addplot[mark = none, smooth, line width = 1pt, color = ForestGreen, domain = 0:' + str(k_def+1) + '] plot coordinates{' + cd_str_3 + '} node[above, pos = 0.75] {\\texttt{LCL = ' + str(round(movAv.Mt_LCL[k+1], 4)) +'}};'
UCL_line        = '\t \t \\addplot[mark = none, smooth, line width = 1pt, color = ForestGreen, domain = 0:' + str(k_def+1) + '] plot coordinates{' + cd_str_4 + '} node[below, pos = 0.75] {\\texttt{UCL = ' + str(round(movAv.Mt_UCL[k+1], 4)) +'}};'

print(before,'\n',str_scatter, '\n',str_viol, '\n',LCL_line, '\n',UCL_line, '\n', after)

\begin{figure}[H] 
 	 \centering 
 	 \begin{tikzpicture} 
 	 	 \begin{axis}[ylabel near ticks, width = 0.75\textwidth, xlabel = Subgroup, ylabel = $M_t$ , grid = both, grid style = {dotted, gray}] 
 	 	 \addplot[only marks, color = blue] plot coordinates{(1, 50.79806) (2, 48.50609) (3, 49.62337) (4, 49.78696) (5, 50.59259) (6, 50.60655) (7, 50.71859) (8, 50.83533) (9, 51.00508)}; 
 	 	 \addplot[only marks, color = red, mark size = 2pt] plot coordinates{(10, 52.05022) (11, 52.2036) (12, 52.79759) (13, 52.85237) (14, 52.82834) (15, 52.69814) (16, 52.6002) (17, 52.58531) (18, 52.41748) (19, 51.79759) (20, 51.44783)}; 
 	 	 \addplot[mark = none, smooth, line width = 1pt, color = ForestGreen, domain = 0:21] plot coordinates{(1, 46.64589803375031) (2, 47.628291754873715) (3, 48.06350832689629) (4, 48.32294901687516) (5, 48.5) (6, 48.63069360623709) (7, 48.73226861790722) (8, 48.81414587743686) (9, 48.81414587743686) (10, 48.81414587743686) (11, 48.81414587743686) (12, 48.81414587743686) (13,

#### exponentially weighted moving average

In [31]:
movAv               = pd.DataFrame(np.reshape(np.loadtxt('moving_average.txt', delimiter=' '), (-1, 2), order='C'), columns=['Xt_bar', 'Mt']).dropna()
k,n                 = 8, 4
meanP, stdP         = 50, math.sqrt(5)
alpha               = 2/9


# movAv           = movAv[~((movAv.index == 2) | (movAv.index == 4))]


movAv['Wt']         = movAv['Xt_bar'] * 0
k_def               = movAv.shape[0]



movAv.Wt[0]         = meanP
for i in range(1, k_def):
    movAv.Wt[i]      = alpha * movAv.Xt_bar[i] + (1 - alpha) * movAv.Wt[i-1]

UCL, LCL        = meanP + 3 * stdP * math.sqrt(alpha / (n * (2 - alpha))), meanP - 3 * stdP * math.sqrt(alpha / (n * (2 - alpha)))
print(f'LCL = {LCL} and UCL = {UCL}')

violators       = movAv.loc[(movAv['Wt'] > UCL) | (movAv['Wt'] < LCL)]
inControl       = movAv.loc[(movAv['Wt'] < UCL) & (movAv['Wt'] > LCL)]
coords_1     = list(zip(inControl.index + 1, inControl['Wt']))
cd_str_1     = ' '.join(map(str, coords_1))

coords_2     = list(zip(violators.index + 1, violators['Wt']))
cd_str_2     = ' '.join(map(str, coords_2))


before          = '\\begin{figure}[H] \n \t \\centering \n \t \\begin{tikzpicture} \n \t \t \\begin{axis}[ylabel near ticks, width = 0.75\\textwidth, xlabel = Subgroup, ylabel = $W_t$ , grid = both, grid style = {dotted, gray}]'
after           = '\t \t \\end{axis} \n \t \\end{tikzpicture} \n \\end{figure}'
str_scatter     = '\t \t \\addplot[only marks, color = blue] plot coordinates{' + cd_str_1 +'};'
str_viol        = '\t \t \\addplot[only marks, color = red, mark size = 2pt] plot coordinates{' + cd_str_2 +'};'
LCL_line        = '\t \t \\addplot[mark = none, line width = 1pt, color = ForestGreen, domain = 0:' + str(k_def+1) + ']{' + str(round(LCL, 2)) + '} node[above, pos = 0.5] {\\texttt{LCL = ' + str(round(LCL, 4)) +'}};'
UCL_line        = '\t \t \\addplot[mark = none, line width = 1pt, color = ForestGreen, domain = 0:' + str(k_def+1) + ']{' + str(round(UCL, 2)) + '} node[below, pos = 0.5] {\\texttt{UCL = ' + str(round(UCL, 4)) +'}};'

print(before,'\n',str_scatter, '\n',str_viol, '\n',LCL_line, '\n',UCL_line, '\n', after)

LCL = 48.81414587743686 and UCL = 51.18585412256314
\begin{figure}[H] 
 	 \centering 
 	 \begin{tikzpicture} 
 	 	 \begin{axis}[ylabel near ticks, width = 0.75\textwidth, xlabel = Subgroup, ylabel = $W_t$ , grid = both, grid style = {dotted, gray}] 
 	 	 \addplot[only marks, color = blue] plot coordinates{(1, 50.0) (2, 49.15869555555556) (3, 49.75852543209877) (4, 49.87389978052127) (5, 50.74972649596099) (6, 50.733420607969656) (7, 50.8795115839764) (8, 51.05127789864831)}; 
 	 	 \addplot[only marks, color = red, mark size = 2pt] plot coordinates{(9, 51.29678725450424) (10, 52.025330086836625) (11, 52.26080562309515) (12, 52.876111040185116) (13, 53.18217080903287) (14, 52.58258840702557) (15, 52.08629764990878) (16, 51.81577817215127) (17, 51.86492746722877) (18, 52.168843585622376) (19, 51.27040723326185) (20, 51.483992292536996)}; 
 	 	 \addplot[mark = none, line width = 1pt, color = ForestGreen, domain = 0:21]{48.81} node[above, pos = 0.5] {\texttt{LCL = 48.8141}}; 
 	 	 \addplot[

#### cumulative sum control chart

In [48]:
movAv               = pd.DataFrame(np.reshape(np.loadtxt('moving_average.txt', delimiter=' '), (-1, 2), order='C'), columns=['Xt_bar', 'Mt']).dropna()
k,n                 = 8, 4
meanP, stdP         = 30, math.sqrt(40)
alpha               = 2/9
d, B                = 0.5, 4.77

# movAv           = movAv[~((movAv.index == 2) | (movAv.index == 4))]
movAv['Y_i']        = movAv['Xt_bar'] - meanP - (d * stdP / math.sqrt(n))
movAv['W_i']        = -movAv['Xt_bar'] + meanP - (d * stdP / math.sqrt(n))

movAv['St'],  movAv['Tt']        = movAv['Xt_bar'] * 0, movAv['Xt_bar'] * 0
k_def               = movAv.shape[0]



movAv.St[0], movAv.Tt[0]         = 0, 0
for i in range(1, k_def):
    movAv.St[i], movAv.Tt[i]      = max(0, movAv.St[i-1] + movAv.Y_i[i]), max(0, movAv.Tt[i-1] + movAv.W_i[i])

UCL                 = B * stdP / math.sqrt(n)
print(f'UCL = {UCL}')

violators       = movAv.loc[(movAv['St'] > UCL) | (movAv['Tt'] > UCL)]
inControl       = movAv.loc[(movAv['St'] < UCL) & (movAv['Tt'] < UCL)]

coords_1     = list(zip(inControl.index + 1, inControl['St']))
cd_str_1     = ' '.join(map(str, coords_1))

coords_2     = list(zip(violators.index + 1, violators['St']))
cd_str_2     = ' '.join(map(str, coords_2))

coords_3     = list(zip(inControl.index + 1, inControl['Tt']))
cd_str_3     = ' '.join(map(str, coords_3))

coords_4     = list(zip(violators.index + 1, violators['Tt']))
cd_str_4     = ' '.join(map(str, coords_4))


before          = '\\begin{figure}[H] \n \t \\centering \n \t \\begin{tikzpicture} \n \t \t \\begin{axis}[ylabel near ticks, width = 0.75\\textwidth, xlabel = Subgroup, ylabel = $S_i\\ \\text{and}\\ T_i$ , grid = both, grid style = {dotted, gray}]'
after           = '\t \t \\end{axis} \n \t \\end{tikzpicture} \n \\end{figure}'
str_scatter     = '\t \t \\addplot[thick, only marks, mark = o, color = blue] plot coordinates{' + cd_str_1 +'};'
str_viol        = '\t \t \\addplot[thick, only marks, mark = o, color = red, mark size = 2pt] plot coordinates{' + cd_str_2 +'};'
str_scatter_1     = '\t \t \\addplot[thick, only marks, mark = x, color = Violet] plot coordinates{' + cd_str_3 +'};'
str_viol_1        = '\t \t \\addplot[thick, only marks, mark = x, color = BrickRed, mark size = 2pt] plot coordinates{' + cd_str_4 +'};'
UCL_line        = '\t \t \\addplot[mark = none, line width = 1pt, color = ForestGreen, domain = 0:' + str(k_def+1) + ']{' + str(round(UCL, 2)) + '} node[below, pos = 0.5] {\\texttt{UCL = ' + str(round(UCL, 4)) +'}};'

print(before,'\n',str_scatter, '\n',str_viol, '\n',str_scatter_1, '\n',str_viol_1, '\n',UCL_line, '\n', after)

UCL = 15.08406443900317
\begin{figure}[H] 
 	 \centering 
 	 \begin{tikzpicture} 
 	 	 \begin{axis}[ylabel near ticks, width = 0.75\textwidth, xlabel = Subgroup, ylabel = $S_i\ \text{and}\ T_i$ , grid = both, grid style = {dotted, gray}] 
 	 	 \addplot[thick, only marks, mark = o, color = blue] plot coordinates{(1, 0.0) (2, 4.228311169915808) (3, 11.777352339831621) (4, 11.10981350974743) (5, 8.98841467966324) (6, 7.9555658495790516) (7, 8.961627019494864) (8, 13.774628189410674) (9, 12.253899359326486) (10, 8.299790529242298) (11, 3.2621816991581083) (12, 5.70728286907392) (13, 11.878134038989728) (14, 8.113285208905538) (15, 3.413426378821348) (16, 0.0) (17, 0.899561169915809) (18, 1.7654523398316173) (19, 0.0) (20, 6.953191169915807) (21, 9.405822339831614) (22, 6.361663509747424) (23, 7.712264679663232) (24, 4.788375849579043) (25, 5.392707019494855)}; 
 	 	 \addplot[thick, only marks, mark = o, color = red, mark size = 2pt] plot coordinates{}; 
 	 	 \addplot[thick, only marks, mar

In [36]:
inControl

,Xt_bar,Mt,Y_i,W_i,St,Tt
0,50.79806,50.79806,-0.319974,-1.916094,0.000000,0.000000
1,46.21413,48.50609,-4.903904,2.667836,0.000000,2.667836
2,51.85793,49.62337,0.739896,-2.975964,0.739896,0.000000
3,50.27771,49.78696,-0.840324,-1.395744,0.000000,0.000000
4,53.81512,50.59259,2.697086,-4.933154,2.697086,0.000000
5,50.67635,50.60655,-0.441684,-1.794384,2.255402,0.000000
6,51.39083,50.71859,0.272796,-2.508864,2.528198,0.000000
